In [32]:
import tkinter as tk
import random
from copy import deepcopy
from collections import deque
import random
import math

INF = 2**64
PERFECT_MOVE = [[2, 2**2, 2**3, 2**4],
                [2**8, 2**7, 2**6, 2**5],
                [2**9, 2**10, 2**11, 2**12],
                [2**16, 2**15, 2**14, 2**13]]

In [33]:
def colors(c, i):
    match c:
        case "0":
            match i:
                case "2":
                    return "#fcefe6"
                case "4":
                    return "#f2e8cb"
                case "8":
                    return "#f5b682"
                case "16":
                    return "#f29446"
                case "32":
                    return "#ff775c"
                case "64":
                    return "#e642ce"
                case "128":
                    return "#ede291"
                case "256":
                    return "#fce130"
                case "512":
                    return "#ffdb4a"
                case "1024":
                    return "#f0b922"
                case "2048":
                    return "#fad74d"
        
        case "1":
            match i:
                case "2":
                    return "#695c57"
                case "4":
                    return "#695c57"
                case "8":
                    return "#ffffff"
                case "16":
                    return "#ffffff"
                case "32":
                    return "#ffffff"
                case "64":
                    return "#ffffff"
                case "128":
                    return "#ffffff"
                case "256":
                    return "#ffffff"
                case "512":
                    return "#ffffff"
                case "1024":
                    return "#ffffff"
                case "2048":
                    return "#ffffff"
        
        case "2":
            match i:
                case "2":
                    return ("Helvetica", 55, "bold")
                case "4":
                    return ("Helvetica", 55, "bold")
                case "8":
                    return ("Helvetica", 55, "bold")
                case "16":
                    return ("Helvetica", 50, "bold")
                case "32":
                    return ("Helvetica", 50, "bold")
                case "64":
                    return ("Helvetica", 50, "bold")
                case "128":
                    return ("Helvetica", 45, "bold")
                case "256":
                    return ("Helvetica", 45, "bold")
                case "512":
                    return ("Helvetica", 45, "bold")
                case "1024":
                    return ("Helvetica", 40, "bold")
                case "2048":
                    return ("Helvetica", 40, "bold")
        

### Manual Play. If you want to try out the environment without AI interference, this is the game

In [34]:
# For manual playing if u want to have some fun

class Game2048(tk.Frame):
    def __init__(self):
        tk.Frame.__init__(self)
        self.grid()
        self.master.title("2048")
        self.game_over_flag = False
        
        #main grid where parent is itself
        self.main_grid = tk.Frame(self, bg = "#a39489", bd=3, width = 600, height = 600)
        self.main_grid.grid(pady=(100, 0))
        self.make_GUI()
        self.starting()
        
        self.master.bind("<Left>", self.left_G)
        self.master.bind("<Right>", self.right_G)
        self.master.bind("<Up>", self.up_G)
        self.master.bind("<Down>", self.down_G)
        
        self.mainloop()
    
    def make_GUI(self):
        # Setting up the grid
        self.cells = []
        for i in range(4):
            row = []
            for j in range(4):
                cell_frame = tk.Frame(self.main_grid, bg="#c2b3a9", width = 150, height = 150)
                cell_frame.grid(row = i, column = j, padx = 5, pady = 5)
                cell_number = tk.Label(self.main_grid, bg = "#c2b3a9")
                cell_number.grid(row=i, column = j)
                cell_data = {"frame": cell_frame, "number": cell_number}
                row.append(cell_data)
            self.cells.append(row)
        
        #make scoring header
        score_frame = tk.Frame(self)
        score_frame.place(relx=0.5, y=45, anchor="center")
        tk.Label(score_frame, text="Score", font = ("Verdana", 24)).grid(row=0)
        self.score_label = tk.Label(score_frame, text="0", font = ("Helvetica", 36, "bold"))
        self.score_label.grid(row=1)
    
    def starting(self):
        #create a matrix of zeros
        self.matrix = [[0] * 4 for _ in range(4)]
        
        #Fill 1 random cells with 2s
        row = random.randint(0,3)
        col = random.randint(0,3)
        self.matrix[row][col] = 2
        self.cells[row][col]["frame"].configure(bg = colors("0","2")) #displaying 2 on cell frame and number
        self.cells[row][col]["number"].configure(
            bg = colors("0","2"),
            fg = colors("1","2"),
            font = colors("2","2"),
            text = "2"
        )
        while(self.matrix[row][col] != 0):
            row = random.randint(0,3)
            col = random.randint(0,3)
        self.matrix[row][col] = 4
        self.cells[row][col]["frame"].configure(bg = colors("0","4")) #displaying 4 on cell frame and number
        self.cells[row][col]["number"].configure(
            bg = colors("0","4"),
            fg = colors("1","4"),
            font = colors("2","4"),
            text = "4"
        )
        
        self.score = 0
    
    
    #compress all non-zero numbers to one side of the board
    
    def stack(self):
        new_matrix = [[0] * 4 for _ in range(4)]
        for i in range(4):
            fill_pos = 0
            for j in range(4):
                if self.matrix[i][j] != 0:
                    new_matrix[i][fill_pos] = self.matrix[i][j]
                    fill_pos += 1
        self.matrix = new_matrix
    
    
    #tiles merges to 1 value
    def combine(self):
        for i in range(4):
            for j in range(3):
                if self.matrix[i][j] != 0 and self.matrix[i][j] == self.matrix[i][j+1]:
                    self.matrix[i][j] *= 2
                    self.matrix[i][j+1] = 0
                    self.score += self.matrix[i][j]
    
    #reverse the matrix
    def reverse(self):
        new_matrix = []
        for i in range(4):
            new_matrix.append([])
            for j in range(4):
                new_matrix[i].append(self.matrix[i][3-j]) #order of values are reversed
        self.matrix = new_matrix
    
    
    def transpose(self):
        new_matrix = [[0] * 4 for _ in range(4)]
        for i in range(4):
            for j in range(4):
                new_matrix[i][j] = self.matrix[j][i]
        self.matrix = new_matrix
        
    #Add a new 2 or 4 tile randomly to an empty cell
    def adding_tiles(self):
        row = random.randint(0,3)
        col = random.randint(0,3)
        while(self.matrix[row][col] != 0):
            row = random.randint(0,3)
            col = random.randint(0,3)
        if random.random() < 0.9:
            self.matrix[row][col] = 2
        else:
            self.matrix[row][col] = 4
        
    
    def can_move_left(self):
        for row in self.matrix:
            for i in range(len(row)-1, 0, -1):
                if row[i] != 0 and (row[i] == row[i - 1] or row[i-1] == 0):
                    return True
        return False

    def can_move_right(self):
        for row in self.matrix:
            for i in range(len(row)-1):
                if row[i] != 0 and (row[i] == row[i + 1] or row[i+1] == 0):
                    return True
        return False

    def can_move_up(self):
        for col in range(4):
            for i in range(3, 0, -1):
                if self.matrix[i][col] != 0 and (self.matrix[i][col] == self.matrix[i-1][col] or self.matrix[i-1][col] == 0):
                    return True
        return False

    def can_move_down(self):
        for col in range(4):
            for i in range(3):  
                if self.matrix[i][col] != 0 and (self.matrix[i][col] == self.matrix[i+1][col] or self.matrix[i+1][col] == 0):
                    return True
        return False
    
    
    
    def perform_move(self, move):
        got_move = -1
        if move == "left":
            if self.can_move_left():
                self.stack()
                self.combine()
                self.stack()
                got_move = 1
        elif move == "right":
            if self.can_move_right():
                self.reverse()
                self.stack()
                self.combine()
                self.stack()
                self.reverse()
                got_move = 1
        elif move == "up":
            if self.can_move_up():
                self.transpose()
                self.stack()
                self.combine()
                self.stack()
                self.transpose()
                got_move = 1
        elif move == "down":
            if self.can_move_down():
                self.transpose()
                self.reverse()
                self.stack()
                self.combine()
                self.stack()
                self.reverse()
                self.transpose()
                got_move = 1
                
        if got_move == 1:
            self.adding_tiles()
            self.game_over()
            got_move = -1
            
    
    #update GUI to match the matrix
    def update_GUI(self):
        for i in range(4):
            for j in range(4):
                cell_value = self.matrix[i][j]
                if cell_value == 0:
                    self.cells[i][j]["frame"].configure(bg = "#c2b3a9")
                    self.cells[i][j]["number"].configure(bg = "#c2b3a9", text="")
                else:
                    self.cells[i][j]["frame"].configure(bg = colors("0", str(cell_value)))
                    self.cells[i][j]["number"].configure(
                        bg = colors("0", str(cell_value)),
                        fg = colors("1", str(cell_value)),
                        font = colors("2", str(cell_value)),
                        text = str(cell_value)
                    )
        self.score_label.configure(text = self.score)
        self.update_idletasks()
    
    def exit_game(self):
        self.master.quit()
        self.master.destroy()
    
    #Arrow functions
    def left_G(self, event):
        if not self.game_over_flag:
            self.perform_move("left")
            self.update_GUI()
        else:
            self.exit_game()
        
    
    def right_G(self, event):
        if not self.game_over_flag:
            self.perform_move("right")
            self.update_GUI()
        else:
            self.exit_game()
    
    def up_G(self, event):
        if not self.game_over_flag:
            self.perform_move("up")
            self.update_GUI()
        else:
            self.exit_game()
        
    def down_G(self, event):
        if not self.game_over_flag:
            self.perform_move("down")
            self.update_GUI()
        else:
            self.exit_game()

    
    #check if any moves are left avaliable
    def horizontal_moves_left(self):
        for i in range(4):
            for j in range(3):
                if self.matrix[i][j] == self.matrix[i][j+1]:
                    return True
        return False
    
    def vertical_moves_left(self):
        for i in range(3):
            for j in range(4):
                if self.matrix[i][j] == self.matrix[i+1][j]:
                    return True
        return False
    
    #check whether the game is over
    def game_over(self):
        if any(2048 in row for row in self.matrix):
            game_over_frame = tk.Frame(self.main_grid, borderwidth = 2)
            game_over_frame.place(relx = 0.5, rely = 0.5, anchor="center")
            tk.Label(
                game_over_frame,
                text = "You Win!",
                bg = "#ffcc00",
                fg = "#ffffff",
                font = ("Helvetica", 48, "bold")
            ).pack()
            self.game_over_flag = True
            
        elif not any(0 in row for row in self.matrix) and not self.horizontal_moves_left() and not self.vertical_moves_left():
            game_over_frame = tk.Frame(self.main_grid, borderwidth = 2)
            game_over_frame.place(relx = 0.5, rely = 0.5, anchor="center")
            tk.Label(
                game_over_frame,
                text = "Game Over!",
                bg = "#a39489",
                fg = "#ffffff",
                font = ("Helvetica", 48, "bold")
            ).pack()
            self.game_over_flag = True

In [35]:
def main():
    Game2048()

if __name__ == "__main__":
    main()

### Game logic, which includes all the possible functions for the 2048 game without the interference of the GUI

In [36]:
#All the game functions without gui interference

class GameLogic:
    def __init__(self):
        self.initialize_board()
        self.score = 0
        self.moves_taken = 0

    def initialize_board(self):
        #create a matrix of zeros
        self.matrix = [[0] * 4 for _ in range(4)]
        
        #Fill 1 random cells with 2s
        row = random.randint(0,3)
        col = random.randint(0,3)
        self.matrix[row][col] = 2
        while(self.matrix[row][col] != 0):
            row = random.randint(0,3)
            col = random.randint(0,3)
        self.matrix[row][col] = 4

    def add_new_tile(self):
        row = random.randint(0,3)
        col = random.randint(0,3)
        while(self.matrix[row][col] != 0):
            row = random.randint(0,3)
            col = random.randint(0,3)
        if random.random() < 0.9:
            self.matrix[row][col] = 2
        else:
            self.matrix[row][col] = 4

    def perform_move_ex(self, move):
        if move == "left":
            if self.can_move_left():
                self.compress()
                self.merge()
                self.compress()
                return 1
            else:
                return 0
        elif move == "right":
            if self.can_move_right():
                self.reverse()
                self.compress()
                self.merge()
                self.compress()
                self.reverse()
                return 1
            else:
                return 0
        elif move == "up":
            if self.can_move_up():
                self.transpose()
                self.compress()
                self.merge()
                self.compress()
                self.transpose()
                return 1
            else:
                return 0
        elif move == "down":
            if self.can_move_down():
                self.transpose()
                self.reverse()
                self.compress()
                self.merge()
                self.compress()
                self.reverse()
                self.transpose()
                return 1
            else:
                return 0
    
    def perform_move(self, move):
        got_move = -1
        if move == "left":
            if self.can_move_left():
                self.compress()
                self.merge()
                self.compress()
                got_move = 1
        elif move == "right":
            if self.can_move_right():
                self.reverse()
                self.compress()
                self.merge()
                self.compress()
                self.reverse()
                got_move = 1
        elif move == "up":
            if self.can_move_up():
                self.transpose()
                self.compress()
                self.merge()
                self.compress()
                self.transpose()
                got_move = 1
        elif move == "down":
            if self.can_move_down():
                self.transpose()
                self.reverse()
                self.compress()
                self.merge()
                self.compress()
                self.reverse()
                self.transpose()
                got_move = 1
                
        if got_move == 1:
            self.add_new_tile()
            self.game_over()
            self.moves_taken += 1
            if self.game_over() == 1:
                return 1
            elif self.game_over() == 0:
                return 0
        
        return got_move


    def compress(self):
        new_matrix = [[0] * 4 for _ in range(4)]
        for i in range(4):
            fill_pos = 0
            for j in range(4):
                if self.matrix[i][j] != 0:
                    new_matrix[i][fill_pos] = self.matrix[i][j]
                    fill_pos += 1
        self.matrix = new_matrix

    def merge(self):
        for i in range(4):
            for j in range(3):
                if self.matrix[i][j] != 0 and self.matrix[i][j] == self.matrix[i][j+1]:
                    self.matrix[i][j] *= 2
                    self.matrix[i][j+1] = 0
                    self.score += self.matrix[i][j]

    def can_move_left(self):
        for row in self.matrix:
            for i in range(len(row)-1, 0, -1):
                if row[i] != 0 and (row[i] == row[i - 1] or row[i-1] == 0):
                    return True
        return False
    
    def can_move_right(self):
        for row in self.matrix:
            for i in range(len(row) - 1):  
                if row[i] != 0 and (row[i] == row[i + 1] or row[i + 1] == 0):  
                    return True
        return False

    def can_move_up(self):
        for col in range(4):
            for i in range(3, 0, -1):
                if self.matrix[i][col] != 0 and (self.matrix[i][col] == self.matrix[i-1][col] or self.matrix[i-1][col] == 0):
                    return True
        return False
    
    def can_move_down(self):
        for col in range(4):
            for i in range(3):  
                if self.matrix[i][col] != 0 and (self.matrix[i][col] == self.matrix[i+1][col] or self.matrix[i+1][col] == 0):
                    return True
        return False

    def transpose(self):
        new_matrix = [[0] * 4 for _ in range(4)]
        for i in range(4):
            for j in range(4):
                new_matrix[i][j] = self.matrix[j][i]
        self.matrix = new_matrix
    
    def reverse(self):
        new_matrix = []
        for i in range(4):
            new_matrix.append([])
            for j in range(4):
                new_matrix[i].append(self.matrix[i][3-j]) #order of values are reversed
        self.matrix = new_matrix
        
    
    def horizontal_moves_left(self):
        for i in range(4):
            for j in range(3):
                if self.matrix[i][j] == self.matrix[i][j+1]:
                    return True
        return False
    
    def vertical_moves_left(self):
        for i in range(3):
            for j in range(4):
                if self.matrix[i][j] == self.matrix[i+1][j]:
                    return True
        return False
    
    
    def find_largest_tile(self):
        largest_tile = 0
        for row in self.matrix:
            for tile in row:
                largest_tile = max(largest_tile, tile)
        return largest_tile
    
    
    def game_over(self):
        # Check for game over conditions (no empty cells and no possible merges)
        if any(2048 in row for row in self.matrix):
            return 1
        elif not any(0 in row for row in self.matrix) and not self.horizontal_moves_left() and not self.vertical_moves_left():
            return 0
        else:
            return -1

    def get_score(self):
        return self.score
    
    def get_moves(self):
        return self.moves_taken

    def evaluate_moves(self):
        moves = ['left', 'right', 'up', 'down']
        best_move = None
        best_score = float('-inf')

        for move in moves:
            copied_game = deepcopy(self)
            getattr(copied_game, move)()  # Simulate the move

            # Evaluate the resulting board state using a custom scoring function
            score = self.evaluate_board_state(copied_game.matrix)

        
            if score > best_score:
                best_move = move
                best_score = score

        
        best_move_function = getattr(self, best_move) # Choose based on evaluation
        return best_move_function
        
    
    def evaluate_board_state(self):
        subX = -INF
        subY = -INF
        score = 0
        snake = 1
        sub_snake = 1
        turn=0
        tile = INF

         # Prioritize placing largest tiles in the bottom-left corner
        for i in range(3, -1, -1):
            if turn==0:
                for j in range(4):
                    if self.matrix[3][0] == self.find_largest_tile(): #incentivise the state for longer chain length
                        
                        if tile >= self.matrix[i][j] and self.matrix[i][j] != 0 and snake == 1:
                            score += 3**(math.log2(self.matrix[i][j])-1)
                            
                        elif sub_snake == 1 and tile >= self.matrix[i-1][j] and not i==0:
                            subX = i-1
                            subY = j     #Place a marker on the spot where sub chain can be formed
                            sub_snake = 0
                            
                        elif i == subX and j == subY and self.matrix[i][j] != 0:
                            snake = 1
                            score += 3**(math.log2(self.matrix[i][j])-1) #look back to initial if function to find another chain
                        
                        else:
                            snake = 0
                            sub_snake = 0
                    
                    tile = self.matrix[i][j]
                    if tile != 0:
                        score += self.matrix[i][j] * PERFECT_MOVE[i][j]
                    turn = 1
            else:
                for j in range(3, -1, -1):
                    if self.matrix[3][0] == self.find_largest_tile(): #incentivise the state for longer chain length
                        
                        if tile >= self.matrix[i][j] and self.matrix[i][j] != 0 and snake == 1:
                            score += 3**(math.log2(self.matrix[i][j])-1)
                            
                        elif sub_snake == 1 and tile >= self.matrix[i-1][j] and not i==0:
                            subX = i-1
                            subY = j     #Place a marker on the spot where sub chain can be formed
                            sub_snake = 0
                            
                        elif i == subX and j == subY and self.matrix[i][j] != 0:
                            snake = 1
                            score += 3**(math.log2(self.matrix[i][j])-1) #look back to initial if function to find another chain
                       
                        else:
                            snake = 0
                            sub_snake = 0
                    
                    tile = self.matrix[i][j]
                    if tile != 0:
                        score += self.matrix[i][j] * PERFECT_MOVE[i][j]
                    turn = 0
        
        return score

In [ ]:
#Theoritical usage of bfs search. PLEASE DONT RUN it as it will crash the system due to the amount of state it is processing

def bfs_search(game_logic):
    queue = deque([(game_logic)])
    total_moves = 0
    total_score = 0
    state = -1

    while queue:
        current_game = queue.popleft()

        if state == 1:  # Assuming is_2048 checks if the game state has reached 2048
            total_moves = new_game.get_moves()
            total_score = new_game.get_score()
            break  # Exit the loop once 2048 is reached

        for move in ["left", "right", "up", "down"]:
            new_game = deepcopy(current_game)  # Create a new instance for each move
            state = new_game.perform_move(move)
            if(state == 1):
                break
            elif(state == 0):
                continue
            queue.append((new_game))

    return total_moves, total_score

# Example usage:
game_logic = GameLogic()  # Initialize the game logic
total_moves, total_score = bfs_search(game_logic)

print(f"Total moves taken to reach 2048: {total_moves}")
print(f"Total score at 2048: {total_score}")

In [ ]:
#Theoritical usage of dfs search. PLEASE DONT RUN it as it will crash the system due to the amount of state it is processing

def dfs_search(game_logic, max_depth=1000):
    stack = [(game_logic, 0)]  
    total_moves = 0
    total_score = 0
    state = -1

    while stack:
        current_game, depth = stack.pop()

        if state == 1:  
            total_moves = new_game.get_moves()
            total_score = new_game.get_score()
            print(f"Move: {move}")
            print("Matrix:")
            for row in new_game.matrix:
                print(row)
            print("Score:", new_game.get_score())
            print()
            break  

        if depth < max_depth:  
            for move in ["left", "right", "up", "down"]:
                new_game = deepcopy(current_game) 
                state = new_game.perform_move(move)
                if(state == 1):
                    break
                elif(state == 0):
                    continue
                    
                print(f"Move: {move}")
                print("Matrix:")
                for row in new_game.matrix:
                    print(row)
                print("Score:", new_game.get_score())
                print()
                stack.append((new_game, depth + 1))  
                
    return total_moves, total_score


game_logic = GameLogic()  
total_moves, total_score = dfs_search(game_logic)

print(f"Total moves taken to reach 2048: {total_moves}")
print(f"Total score at 2048: {total_score}")

In [ ]:
#Theoritical usage of ucs search. PLEASE DONT RUN it as \all moves are equal cost. no frontier to compare. 
#This essentially implement DFS but with extra steps.

import heapq

def uniform_cost_search(game_logic):
    queue = [(0, game_logic)]  
    visited = set()  
    total_moves = 0
    total_score = 0
    state = -1

    while queue:
        cost, current_game = heapq.heappop(queue)

        if state == 1:
            break

        if current_game.matrix not in visited:
            visited.add(tuple(map(tuple, current_game.matrix)))  # Convert matrix to tuple for hashing

            for move in ["left", "right", "up", "down"]:
                new_game = deepcopy(current_game)
                new_game.perform_move(move)
                if tuple(map(tuple, new_game.matrix)) not in visited:
                    cost = new_game.get_moves #cost of search would be the moves taken
                    heapq.heappush(queue, (cost, new_game))
                if new_game.game_over() == 1:
                    total_moves = new_game.get_moves()
                    total_score = new_game.get_score()
                    print(f"Final move: {move}")
                    print("Matrix:")
                    for row in current_game.matrix:
                        print(row)
                    print("Score:", current_game.get_score())
                    print()
                    state = 1
                    break
                elif new_game.game_over() == 0:
                    continue
                

    return total_moves, total_score


game_logic = GameLogic()  
total_moves, total_score = uniform_cost_search(game_logic)

print(f"Total moves taken to reach 2048: {total_moves}")
print(f"Total score at 2048: {total_score}")

### Greedy Search (results is suboptimal)

In [37]:
#Based off of which one gives highest score or whatever heuristic function we wanna use. Might not give the desired result
#since it only considers the next best move, not the implications behind it


def greedy_search(game):
    max_score = -INF
    best_move = None
    
    for move in ["left", "right", "up", "down"]:
        new_game = deepcopy(game)
        state = new_game.perform_move_ex(move)
        if state == 0:
            continue
        score = new_game.evaluate_board_state()
        if score > max_score:
            max_score = score
            best_move = move
        
    return best_move

### Expectimax search (most optimal result)

In [14]:
#Best algorithm thus far to solve the 2048 game, considering it make use of decision making with uncertainty
#Right now the algorithm is running at depth 3, (player -> chance -> player)

def expectimax_search(game):
    state = -1
    max_score = float('-inf')
    chance_max = float('-inf')
    total_score = 0
    best_move = None   
    
    for move in ["left", "right", "up", "down"]:
        new_game = deepcopy(game)
        state = new_game.perform_move_ex(move)
        if state == 0:
            continue
        empty_cells = [(i, j) for i in range(4) for j in range(4) if new_game.matrix[i][j] == 0] #chance nodes
        tiles = len(empty_cells)
        for i, j in empty_cells:
            chance_game = deepcopy(new_game)
            chance_game.matrix[i][j] = 2
            chance_max = float('-inf')
            for moves in ["left", "right", "up", "down"]:
                player_game = deepcopy(chance_game)
                state = player_game.perform_move_ex(moves) #Player selection of the best move from the chance node
                if state == 0:
                    continue
                chance_score = player_game.evaluate_board_state()
                if chance_score > chance_max:
                    chance_max = chance_score
                
            total_score += chance_max
            chance_game.matrix[i][j] = 0

        avg_score = total_score / tiles
        if avg_score > max_score: #highest average across all chance nodes
            max_score = avg_score
            best_move = move
        
        avg_score = 0
        total_score = 0
    
    return best_move

In [8]:
# Model for expectimax without GUI (printed using array of matrix). No need to run if u want to see workable GUI below this

def expectimax(game_logic):
    best_move = None
    total_moves = 0
    total_score = 0

    game = deepcopy(game_logic)
    while not game.game_over() == 1:
        best_move = expectimax_search(game)
        game.perform_move(best_move)
        print(f"Move: {best_move}")
        print("Matrix:")
        for row in game.matrix:
            print(row)
        print("Score:", game.get_score())
        print()
        if game.game_over() == 0:
            print("ur AI sucks")
            return -1, -1

    total_moves = game.get_moves()
    total_score = game.get_score()

    return total_moves, total_score


game_logic = GameLogic()  
total_moves, total_score = expectimax(game_logic)

print(f"Total moves taken to reach 2048: {total_moves}")
print(f"Total score at 2048: {total_score}")

Move: left
Matrix:
[2, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[4, 0, 2, 0]
Score: 0

Move: left
Matrix:
[2, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[4, 2, 0, 2]
Score: 0

Move: left
Matrix:
[2, 0, 0, 0]
[0, 0, 0, 0]
[2, 0, 0, 0]
[4, 4, 0, 0]
Score: 4

Move: down
Matrix:
[0, 0, 2, 0]
[0, 0, 0, 0]
[4, 0, 0, 0]
[4, 4, 0, 0]
Score: 8

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 2, 0]
[0, 0, 0, 0]
[8, 4, 2, 0]
Score: 16

Move: down
Matrix:
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 0, 0, 0]
[8, 4, 4, 0]
Score: 20

Move: left
Matrix:
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 0, 2, 0]
[8, 8, 0, 0]
Score: 28

Move: down
Matrix:
[0, 0, 0, 2]
[0, 0, 0, 0]
[2, 0, 0, 0]
[8, 8, 2, 0]
Score: 28

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 0, 0]
[2, 4, 0, 0]
[8, 8, 2, 2]
Score: 28

Move: left
Matrix:
[0, 0, 0, 0]
[0, 2, 0, 0]
[2, 4, 0, 0]
[16, 4, 0, 0]
Score: 48

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 0, 2]
[2, 2, 0, 0]
[16, 8, 0, 0]
Score: 56

Move: down
Matrix:
[2, 0, 0, 0]
[0, 0, 0, 0]
[2, 2, 0, 0]
[16, 8, 0, 2]
Score: 56

Move: left
Matrix

Move: right
Matrix:
[0, 0, 0, 2]
[0, 0, 0, 4]
[2, 0, 2, 32]
[128, 64, 4, 2]
Score: 1184

Move: right
Matrix:
[0, 0, 0, 2]
[2, 0, 0, 4]
[0, 0, 4, 32]
[128, 64, 4, 2]
Score: 1188

Move: right
Matrix:
[0, 0, 0, 2]
[2, 0, 2, 4]
[0, 0, 4, 32]
[128, 64, 4, 2]
Score: 1188

Move: right
Matrix:
[0, 0, 0, 2]
[2, 0, 4, 4]
[0, 0, 4, 32]
[128, 64, 4, 2]
Score: 1192

Move: down
Matrix:
[0, 0, 0, 2]
[0, 0, 0, 4]
[2, 2, 4, 32]
[128, 64, 8, 2]
Score: 1200

Move: right
Matrix:
[0, 0, 0, 2]
[2, 0, 0, 4]
[0, 4, 4, 32]
[128, 64, 8, 2]
Score: 1204

Move: right
Matrix:
[0, 0, 0, 2]
[2, 0, 2, 4]
[0, 0, 8, 32]
[128, 64, 8, 2]
Score: 1212

Move: right
Matrix:
[0, 0, 0, 2]
[0, 0, 4, 4]
[0, 2, 8, 32]
[128, 64, 8, 2]
Score: 1216

Move: right
Matrix:
[0, 0, 0, 2]
[0, 0, 4, 8]
[0, 2, 8, 32]
[128, 64, 8, 2]
Score: 1224

Move: down
Matrix:
[4, 0, 0, 2]
[0, 0, 0, 8]
[0, 2, 4, 32]
[128, 64, 16, 2]
Score: 1240

Move: down
Matrix:
[0, 0, 2, 2]
[0, 0, 0, 8]
[4, 2, 4, 32]
[128, 64, 16, 2]
Score: 1240

Move: left
Matrix:
[4,

Move: right
Matrix:
[0, 0, 2, 4]
[2, 0, 2, 4]
[0, 0, 0, 16]
[256, 128, 64, 8]
Score: 2872

Move: down
Matrix:
[0, 0, 2, 0]
[0, 0, 0, 8]
[2, 0, 4, 16]
[256, 128, 64, 8]
Score: 2884

Move: down
Matrix:
[0, 0, 0, 0]
[2, 0, 2, 8]
[2, 0, 4, 16]
[256, 128, 64, 8]
Score: 2884

Move: right
Matrix:
[0, 0, 2, 0]
[0, 0, 4, 8]
[0, 2, 4, 16]
[256, 128, 64, 8]
Score: 2888

Move: right
Matrix:
[4, 0, 0, 2]
[0, 0, 4, 8]
[0, 2, 4, 16]
[256, 128, 64, 8]
Score: 2888

Move: down
Matrix:
[0, 2, 0, 2]
[0, 0, 0, 8]
[4, 2, 8, 16]
[256, 128, 64, 8]
Score: 2896

Move: down
Matrix:
[2, 0, 0, 2]
[0, 0, 0, 8]
[4, 4, 8, 16]
[256, 128, 64, 8]
Score: 2900

Move: right
Matrix:
[0, 4, 0, 4]
[0, 0, 0, 8]
[0, 8, 8, 16]
[256, 128, 64, 8]
Score: 2912

Move: right
Matrix:
[0, 0, 0, 8]
[0, 4, 0, 8]
[0, 0, 16, 16]
[256, 128, 64, 8]
Score: 2936

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 2, 16]
[0, 4, 16, 16]
[256, 128, 64, 8]
Score: 2952

Move: down
Matrix:
[0, 0, 0, 0]
[0, 4, 2, 0]
[0, 4, 16, 32]
[256, 128, 64, 8]
Score: 2984

M

Move: right
Matrix:
[0, 0, 0, 4]
[2, 2, 4, 2]
[0, 0, 16, 2]
[512, 128, 64, 16]
Score: 5164

Move: down
Matrix:
[0, 0, 0, 0]
[2, 0, 4, 4]
[2, 2, 16, 4]
[512, 128, 64, 16]
Score: 5168

Move: down
Matrix:
[0, 2, 0, 0]
[0, 0, 4, 0]
[4, 2, 16, 8]
[512, 128, 64, 16]
Score: 5180

Move: down
Matrix:
[0, 0, 0, 0]
[0, 2, 4, 0]
[4, 4, 16, 8]
[512, 128, 64, 16]
Score: 5184

Move: left
Matrix:
[0, 0, 0, 2]
[2, 4, 0, 0]
[8, 16, 8, 0]
[512, 128, 64, 16]
Score: 5192

Move: right
Matrix:
[0, 2, 0, 2]
[0, 0, 2, 4]
[0, 8, 16, 8]
[512, 128, 64, 16]
Score: 5192

Move: down
Matrix:
[0, 0, 2, 2]
[0, 2, 2, 4]
[0, 8, 16, 8]
[512, 128, 64, 16]
Score: 5192

Move: down
Matrix:
[0, 0, 0, 2]
[2, 2, 4, 4]
[0, 8, 16, 8]
[512, 128, 64, 16]
Score: 5196

Move: right
Matrix:
[0, 2, 0, 2]
[0, 0, 4, 8]
[0, 8, 16, 8]
[512, 128, 64, 16]
Score: 5208

Move: down
Matrix:
[0, 0, 2, 0]
[0, 2, 4, 2]
[0, 8, 16, 16]
[512, 128, 64, 16]
Score: 5224

Move: down
Matrix:
[2, 0, 2, 0]
[0, 2, 4, 0]
[0, 8, 16, 2]
[512, 128, 64, 32]
Score: 5

Move: right
Matrix:
[2, 0, 0, 4]
[0, 0, 8, 2]
[0, 0, 8, 64]
[512, 256, 128, 4]
Score: 6856

Move: down
Matrix:
[0, 2, 0, 4]
[0, 0, 0, 2]
[2, 0, 16, 64]
[512, 256, 128, 4]
Score: 6872

Move: down
Matrix:
[0, 0, 2, 4]
[0, 0, 0, 2]
[2, 2, 16, 64]
[512, 256, 128, 4]
Score: 6872

Move: right
Matrix:
[0, 0, 2, 4]
[0, 0, 0, 2]
[2, 4, 16, 64]
[512, 256, 128, 4]
Score: 6876

Move: left
Matrix:
[2, 4, 0, 2]
[2, 0, 0, 0]
[2, 4, 16, 64]
[512, 256, 128, 4]
Score: 6876

Move: down
Matrix:
[0, 0, 0, 2]
[2, 0, 0, 2]
[4, 8, 16, 64]
[512, 256, 128, 4]
Score: 6888

Move: left
Matrix:
[2, 0, 2, 0]
[4, 0, 0, 0]
[4, 8, 16, 64]
[512, 256, 128, 4]
Score: 6892

Move: down
Matrix:
[0, 0, 0, 0]
[2, 2, 2, 0]
[8, 8, 16, 64]
[512, 256, 128, 4]
Score: 6900

Move: right
Matrix:
[0, 0, 0, 0]
[0, 0, 2, 4]
[2, 16, 16, 64]
[512, 256, 128, 4]
Score: 6920

Move: left
Matrix:
[2, 0, 0, 0]
[2, 4, 0, 0]
[2, 32, 64, 0]
[512, 256, 128, 4]
Score: 6952

Move: right
Matrix:
[0, 2, 0, 2]
[0, 0, 2, 4]
[0, 2, 32, 64]
[512, 256, 128, 

Move: left
Matrix:
[4, 0, 0, 0]
[4, 0, 0, 2]
[4, 2, 16, 0]
[1024, 128, 16, 0]
Score: 9908

Move: down
Matrix:
[2, 0, 0, 0]
[4, 0, 0, 0]
[8, 2, 0, 0]
[1024, 128, 32, 2]
Score: 9948

Move: right
Matrix:
[0, 0, 0, 2]
[0, 0, 2, 4]
[0, 0, 8, 2]
[1024, 128, 32, 2]
Score: 9948

Move: down
Matrix:
[0, 2, 0, 0]
[0, 0, 2, 2]
[0, 0, 8, 4]
[1024, 128, 32, 4]
Score: 9952

Move: right
Matrix:
[0, 0, 2, 2]
[0, 0, 0, 4]
[0, 0, 8, 4]
[1024, 128, 32, 4]
Score: 9956

Move: right
Matrix:
[0, 0, 0, 4]
[2, 0, 0, 4]
[0, 0, 8, 4]
[1024, 128, 32, 4]
Score: 9960

Move: down
Matrix:
[2, 0, 0, 0]
[0, 0, 0, 0]
[2, 0, 8, 8]
[1024, 128, 32, 8]
Score: 9976

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 0, 0]
[4, 2, 8, 0]
[1024, 128, 32, 16]
Score: 9996

Move: right
Matrix:
[0, 4, 0, 0]
[0, 0, 0, 0]
[0, 4, 2, 8]
[1024, 128, 32, 16]
Score: 9996

Move: right
Matrix:
[0, 0, 0, 4]
[0, 2, 0, 0]
[0, 4, 2, 8]
[1024, 128, 32, 16]
Score: 9996

Move: down
Matrix:
[2, 0, 0, 0]
[0, 2, 0, 4]
[0, 4, 2, 8]
[1024, 128, 32, 16]
Score: 9996



Move: right
Matrix:
[0, 2, 0, 2]
[0, 2, 4, 4]
[0, 4, 16, 32]
[1024, 256, 64, 2]
Score: 11296

Move: down
Matrix:
[0, 0, 2, 2]
[0, 4, 4, 4]
[0, 4, 16, 32]
[1024, 256, 64, 2]
Score: 11300

Move: down
Matrix:
[0, 2, 2, 2]
[0, 0, 4, 4]
[0, 8, 16, 32]
[1024, 256, 64, 2]
Score: 11308

Move: down
Matrix:
[0, 2, 2, 2]
[0, 2, 4, 4]
[0, 8, 16, 32]
[1024, 256, 64, 2]
Score: 11308

Move: down
Matrix:
[2, 0, 2, 2]
[0, 4, 4, 4]
[0, 8, 16, 32]
[1024, 256, 64, 2]
Score: 11312

Move: down
Matrix:
[0, 0, 2, 2]
[2, 4, 4, 4]
[2, 8, 16, 32]
[1024, 256, 64, 2]
Score: 11312

Move: left
Matrix:
[4, 0, 0, 2]
[2, 8, 4, 0]
[2, 8, 16, 32]
[1024, 256, 64, 2]
Score: 11324

Move: down
Matrix:
[0, 4, 0, 0]
[4, 0, 4, 2]
[4, 16, 16, 32]
[1024, 256, 64, 2]
Score: 11344

Move: right
Matrix:
[0, 0, 0, 4]
[0, 0, 8, 2]
[2, 4, 32, 32]
[1024, 256, 64, 2]
Score: 11384

Move: left
Matrix:
[4, 0, 4, 0]
[8, 2, 0, 0]
[2, 4, 64, 0]
[1024, 256, 64, 2]
Score: 11448

Move: down
Matrix:
[4, 0, 0, 2]
[8, 2, 0, 0]
[2, 4, 4, 0]
[1024, 256

Move: right
Matrix:
[0, 2, 0, 2]
[0, 2, 16, 8]
[0, 0, 16, 16]
[1024, 512, 16, 4]
Score: 13272

Move: down
Matrix:
[0, 0, 0, 2]
[0, 0, 0, 8]
[2, 4, 16, 16]
[1024, 512, 32, 4]
Score: 13308

Move: left
Matrix:
[2, 0, 0, 0]
[8, 0, 2, 0]
[2, 4, 32, 0]
[1024, 512, 32, 4]
Score: 13340

Move: down
Matrix:
[2, 0, 0, 0]
[8, 0, 0, 2]
[2, 4, 2, 0]
[1024, 512, 64, 4]
Score: 13404

Move: right
Matrix:
[0, 0, 0, 2]
[0, 0, 8, 2]
[4, 2, 4, 2]
[1024, 512, 64, 4]
Score: 13404

Move: down
Matrix:
[0, 0, 2, 0]
[0, 0, 8, 2]
[4, 2, 4, 4]
[1024, 512, 64, 4]
Score: 13408

Move: down
Matrix:
[0, 0, 2, 0]
[0, 0, 8, 2]
[4, 2, 4, 2]
[1024, 512, 64, 8]
Score: 13416

Move: down
Matrix:
[0, 0, 2, 0]
[0, 0, 8, 2]
[4, 2, 4, 4]
[1024, 512, 64, 8]
Score: 13420

Move: right
Matrix:
[2, 0, 0, 2]
[0, 0, 8, 2]
[0, 4, 2, 8]
[1024, 512, 64, 8]
Score: 13428

Move: down
Matrix:
[0, 2, 0, 0]
[0, 0, 8, 0]
[2, 4, 2, 4]
[1024, 512, 64, 16]
Score: 13448

Move: left
Matrix:
[2, 0, 0, 0]
[8, 0, 0, 2]
[2, 4, 2, 4]
[1024, 512, 64, 16]
Sc

Move: down
Matrix:
[2, 0, 0, 4]
[0, 0, 4, 8]
[0, 0, 4, 128]
[1024, 512, 128, 4]
Score: 14592

Move: down
Matrix:
[0, 0, 0, 4]
[2, 0, 0, 8]
[2, 0, 8, 128]
[1024, 512, 128, 4]
Score: 14600

Move: left
Matrix:
[4, 0, 2, 0]
[2, 8, 0, 0]
[2, 8, 128, 0]
[1024, 512, 128, 4]
Score: 14600

Move: down
Matrix:
[0, 2, 0, 0]
[4, 0, 0, 0]
[4, 16, 2, 0]
[1024, 512, 256, 4]
Score: 14876

Move: down
Matrix:
[0, 0, 0, 4]
[0, 2, 0, 0]
[8, 16, 2, 0]
[1024, 512, 256, 4]
Score: 14884

Move: down
Matrix:
[0, 0, 0, 0]
[0, 2, 0, 4]
[8, 16, 2, 0]
[1024, 512, 256, 8]
Score: 14892

Move: left
Matrix:
[0, 0, 0, 2]
[2, 4, 0, 0]
[8, 16, 2, 0]
[1024, 512, 256, 8]
Score: 14892

Move: down
Matrix:
[0, 0, 0, 0]
[2, 4, 0, 2]
[8, 16, 2, 2]
[1024, 512, 256, 8]
Score: 14892

Move: right
Matrix:
[0, 2, 0, 0]
[0, 2, 4, 2]
[0, 8, 16, 4]
[1024, 512, 256, 8]
Score: 14896

Move: down
Matrix:
[0, 0, 2, 0]
[0, 4, 4, 2]
[0, 8, 16, 4]
[1024, 512, 256, 8]
Score: 14900

Move: right
Matrix:
[0, 0, 0, 2]
[0, 2, 8, 2]
[0, 8, 16, 4]
[1024,

Move: right
Matrix:
[2, 0, 0, 0]
[0, 2, 4, 2]
[0, 32, 32, 64]
[1024, 512, 256, 128]
Score: 16108

Move: right
Matrix:
[0, 2, 0, 2]
[0, 2, 4, 2]
[0, 0, 64, 64]
[1024, 512, 256, 128]
Score: 16172

Move: right
Matrix:
[0, 2, 0, 4]
[0, 2, 4, 2]
[0, 0, 0, 128]
[1024, 512, 256, 128]
Score: 16304

Move: down
Matrix:
[0, 0, 0, 0]
[0, 0, 0, 4]
[2, 4, 4, 2]
[1024, 512, 256, 256]
Score: 16564

Move: left
Matrix:
[0, 0, 0, 0]
[4, 2, 0, 0]
[2, 8, 2, 0]
[1024, 512, 512, 0]
Score: 17084

Move: left
Matrix:
[0, 0, 0, 0]
[4, 2, 2, 0]
[2, 8, 2, 0]
[1024, 1024, 0, 0]
Score: 18108

Move: left
Matrix:
[0, 0, 2, 0]
[4, 4, 0, 0]
[2, 8, 2, 0]
[2048, 0, 0, 0]
Score: 20160

Total moves taken to reach 2048: 947
Total score at 2048: 20160


### GUI for AI plays

In [38]:
# GUI for AI running, this helps to visualize the ai movement (is quite fast but you can see the overall performance)

class AIplays(tk.Frame):
    def __init__(self, master = None, game_logic = None, search_function=None):
        super().__init__(master)
        self.master = master
        self.game_logic = game_logic
        self.search_function = search_function
        self.grid()
        self.cells = []
        self.master.title("2048")
       
        #main grid where parent is itself
        self.main_grid = tk.Frame(self, bg = "#a39489", bd=3, width = 600, height = 600)
        self.main_grid.grid(pady=(100, 0))
        self.make_GUI()
        self.update_GUI()
        self.start_game()
    
        self.mainloop()
    
    def make_GUI(self):
        # Setting up the grid
        self.cells = []
        for i in range(4):
            row = []
            for j in range(4):
                cell_frame = tk.Frame(self.main_grid, bg="#c2b3a9", width = 150, height = 150)
                cell_frame.grid(row = i, column = j, padx = 5, pady = 5)
                cell_number = tk.Label(self.main_grid, bg = "#c2b3a9")
                cell_number.grid(row=i, column = j)
                cell_data = {"frame": cell_frame, "number": cell_number}
                row.append(cell_data)
            self.cells.append(row)
        
        #make scoring header
        score_frame = tk.Frame(self)
        score_frame.place(relx=0.5, y=45, anchor="center")
        tk.Label(score_frame, text="Score", font = ("Verdana", 24)).grid(row=0)
        self.score_label = tk.Label(score_frame, text="0", font = ("Helvetica", 36, "bold"))
        self.score_label.grid(row=1)
            
    
    #update GUI to match the matrix
    def update_GUI(self):
        for i in range(4):
            for j in range(4):
                cell_value = self.game_logic.matrix[i][j]
                if cell_value == 0:
                    self.cells[i][j]["frame"].configure(bg = "#c2b3a9")
                    self.cells[i][j]["number"].configure(bg = "#c2b3a9", text="")
                else:
                    self.cells[i][j]["frame"].configure(bg = colors("0", str(cell_value)))
                    self.cells[i][j]["number"].configure(
                        bg = colors("0", str(cell_value)),
                        fg = colors("1", str(cell_value)),
                        font = colors("2", str(cell_value)),
                        text = str(cell_value)
                    )
        self.score_label.configure(text = self.game_logic.get_score())
        self.update_idletasks()
    
    def start_game(self):
        # Start the game using game logic
        while True:
            best_move = self.search_function(self.game_logic)
            self.game_logic.perform_move(best_move)
            self.update_GUI()
            self.master.update()
        
            # Check if the game is over
            if self.game_logic.game_over() != -1:
                break
        
        state = self.game_logic.game_over()
        total_moves = self.game_logic.get_moves()
        if state == 1:
            print(f"Total moves taken to reach 2048: {total_moves}")
            self.game_over(1)
        else:
            print(f"Total moves taken: {total_moves}")
            self.game_over(0)
    
    #check whether the game is over
    def game_over(self, state):
        if state == 1:
            game_over_frame = tk.Frame(self.main_grid, borderwidth = 2)
            game_over_frame.place(relx = 0.5, rely = 0.5, anchor="center")
            tk.Label(
                game_over_frame,
                text = "You Win!\nMoves Taken: {}".format(self.game_logic.get_moves()),
                bg = "#ffcc00",
                fg = "#ffffff",
                font = ("Helvetica", 48, "bold")
            ).pack()
            
        else:
            game_over_frame = tk.Frame(self.main_grid, borderwidth = 2)
            game_over_frame.place(relx = 0.5, rely = 0.5, anchor="center")
            tk.Label(
                game_over_frame,
                text = "Game Over!\nMoves Taken: {}".format(self.game_logic.get_moves()),
                bg = "#a39489",
                fg = "#ffffff",
                font = ("Helvetica", 48, "bold")
            ).pack()

In [41]:
game_logic = GameLogic()
root = tk.Tk()
game = AIplays(master=root, game_logic=game_logic, search_function = expectimax_search)

Total moves taken to reach 2048: 1047


In [43]:
game_logic = GameLogic()
root = tk.Tk()
game = AIplays(master=root, game_logic=game_logic, search_function = greedy_search)

Total moves taken: 219


#### Greedy search focuses on immediate gains and may not always lead to the optimal outcome. This accounts for it not being able to hit higher scores comparable to expectimax search. You may ran a few times to see whether it will give a better result.